In [ ]:
!pip install transformers datasets sentencepiece -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, BartTokenizer, BartForConditionalGeneration, PegasusTokenizer, PegasusForConditionalGeneration
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
import pandas as pd

# Verify the path using the "Files" tab in Colab and copy the path
file_path = "/content/drive/MyDrive/FAIRFRAME/dataset_10000.csv"  # Replace with the copied path
df = pd.read_csv(file_path)

# Display sample data
print("Sample Data:")
print(df.head())

# Check for missing values before dropping
print("Missing values before dropping:")
print(df.isnull().sum())

# Remove null values
df.dropna(inplace=True)

# Check if DataFrame is empty after dropping null values
if df.empty:
    print("DataFrame is empty after dropping null values. Please check your data.")
else:
    # Train-validation split (80%-20%)
    train_texts, val_texts, train_labels, val_labels = train_test_split(df['text'], df['detoxified'], test_size=0.2, random_state=42)

    # Convert to Huggingface Dataset
    train_dataset = Dataset.from_dict({'text': train_texts.tolist(), 'unbiased': train_labels.tolist()})
    val_dataset = Dataset.from_dict({'text': val_texts.tolist(), 'unbiased': val_labels.tolist()})

Sample Data:
                                                text  \
0  McWhorter asserts that the events that Angelou...   
1                  The typical age of onset is 20 40   
2  In John Wells drama Burnt Cooper played a chef...   
3  Simeon turned the new Bulgarian capital Presla...   
4  The Italian Peninsula was left open to all for...   

                                          detoxified  
0  McWhorter suggests that the experiences outlin...  
1     The average age of onset is between 20 and 40.  
2  In the John Wells film "Burnt," actor Bradley ...  
3  Simeon developed the new Bulgarian capital, Pr...  
4  The Italian Peninsula was available for soldie...  
Missing values before dropping:
text            0
detoxified    534
dtype: int64


In [ ]:
# prompt: Dataset({
#     features: ['text', 'unbiased'],
#     num_rows: 1893
# })
# need only the text

# Assuming you have the train_dataset and val_dataset defined as in your provided code

# Extract only the 'text' column from the train_dataset


# Extract only the 'text' column from the val_dataset
val_texts_only = [example['text'] for example in val_dataset]

# Now you have two lists containing only the 'text' data from your train and validation datasets

print( val_texts_only[1],"\n")


There are no historical references to an Aurelius Ursicinus in Britain in this period 



In [ ]:
# prompt: need shape oh val_dataset

val_dataset.text[:5]



AttributeError: 'Dataset' object has no attribute 'text'

In [ ]:
max_input_length = 128
max_target_length = 128

def preprocess_data(data, tokenizer):
    inputs = ["unbias: " + i for i in data['text']]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding='max_length')

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(data['unbiased'], max_length=max_target_length, truncation=True, padding='max_length')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:
# T5 Model
t5_tokenizer = T5Tokenizer.from_pretrained("t5-large")
t5_model = T5ForConditionalGeneration.from_pretrained("t5-large")

# BART Model
'''bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

# Pegasus Model
pegasus_tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-large")
pegasus_model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-large")
'''

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

'bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")\nbart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")\n\n# Pegasus Model\npegasus_tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-large")\npegasus_model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-large")\n'

In [ ]:
train_dataset_t5 = train_dataset.map(lambda x: preprocess_data(x, t5_tokenizer), batched=True, remove_columns=['text', 'unbiased'])
val_dataset_t5 = val_dataset.map(lambda x: preprocess_data(x, t5_tokenizer), batched=True, remove_columns=['text', 'unbiased'])

'''train_dataset_bart = train_dataset.map(lambda x: preprocess_data(x, bart_tokenizer), batched=True, remove_columns=['text', 'unbiased'])
val_dataset_bart = val_dataset.map(lambda x: preprocess_data(x, bart_tokenizer), batched=True, remove_columns=['text', 'unbiased'])

train_dataset_pegasus = train_dataset.map(lambda x: preprocess_data(x, pegasus_tokenizer), batched=True, remove_columns=['text', 'unbiased'])
val_dataset_pegasus = val_dataset.map(lambda x: preprocess_data(x, pegasus_tokenizer), batched=True, remove_columns=['text', 'unbiased'])
'''

Map:   0%|          | 0/7571 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1893 [00:00<?, ? examples/s]

"train_dataset_bart = train_dataset.map(lambda x: preprocess_data(x, bart_tokenizer), batched=True, remove_columns=['text', 'unbiased'])\nval_dataset_bart = val_dataset.map(lambda x: preprocess_data(x, bart_tokenizer), batched=True, remove_columns=['text', 'unbiased'])\n\ntrain_dataset_pegasus = train_dataset.map(lambda x: preprocess_data(x, pegasus_tokenizer), batched=True, remove_columns=['text', 'unbiased'])\nval_dataset_pegasus = val_dataset.map(lambda x: preprocess_data(x, pegasus_tokenizer), batched=True, remove_columns=['text', 'unbiased'])\n"

In [ ]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/FAIRFRAME",
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=3,
    num_train_epochs=3,
    weight_decay=0.05,
    save_total_limit=1,
    logging_steps=50,
    save_strategy="epoch",
    report_to="none"
)

data_collator = DataCollatorForSeq2Seq(tokenizer=t5_tokenizer, model=t5_model)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
def train_model(model, tokenizer, train_dataset, val_dataset):
    train_losses = []  # Initialize empty list for train_losses
    val_losses = []  # Initialize empty list for val_losses
    epochs = [] # Initialize empty list for epochs

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator
    )

    train_results = trainer.train() # capturing train result to access metrics
    return model


# Train T5
print("\nTraining T5...")
t5_model = train_model(t5_model, t5_tokenizer, train_dataset_t5, val_dataset_t5)

#get training loss values
#loss_values=train_results_t5.metrics['train_loss']
#epochs=range(1,len(loss_values)+1)

''' #Train BART
print("\nTraining BART...")
bart_model = train_model(bart_model, bart_tokenizer, train_dataset_bart, val_dataset_bart)

# Train Pegasus
print("\nTraining Pegasus...")
pegasus_model = train_model(pegasus_model, pegasus_tokenizer, train_dataset_pegasus, val_dataset_pegasus)'''



Training T5...


<ipython-input-9-6a799512cb03>:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,0.361300,0.308031
2,0.230300,0.306919
3,0.143100,0.330278


' #Train BART\nprint("\nTraining BART...")\nbart_model = train_model(bart_model, bart_tokenizer, train_dataset_bart, val_dataset_bart)\n\n# Train Pegasus\nprint("\nTraining Pegasus...")\npegasus_model = train_model(pegasus_model, pegasus_tokenizer, train_dataset_pegasus, val_dataset_pegasus)'

In [ ]:
save_path="/content/drive/MyDrive/FAIRFRAME/mitigation_model"
t5_model.save_pretrained(save_path)
t5_tokenizer.save_pretrained(save_path)

('/content/drive/MyDrive/FAIRFRAME/mitigation_model/tokenizer_config.json',
 '/content/drive/MyDrive/FAIRFRAME/mitigation_model/special_tokens_map.json',
 '/content/drive/MyDrive/FAIRFRAME/mitigation_model/spiece.model',
 '/content/drive/MyDrive/FAIRFRAME/mitigation_model/added_tokens.json')

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
model_path="/content/drive/MyDrive/FAIRFRAME/mitigation_model"
mitigation_tokenizer = T5Tokenizer.from_pretrained(model_path)
mitigation_model = T5ForConditionalGeneration.from_pretrained(model_path)

max_input_length = 128
max_target_length = 128

def generate_unbiased_text(text):
    """Generates unbiased text using the T5 model."""
    input_ids = mitigation_tokenizer("Unbias: " + text, return_tensors="pt", padding='max_length', max_length=max_input_length, truncation=True).input_ids
    output_ids = mitigation_model.generate(input_ids, max_length=max_target_length, num_beams=4, early_stopping=True)
    return mitigation_tokenizer.decode(output_ids[0], skip_special_tokens=True)
user_input = input("Enter text: ")
print("Unbiased:", generate_unbiased_text(user_input))

OSError: Error no file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt.index or flax_model.msgpack found in directory /content/drive/MyDrive/FAIRFRAME/mitigation_model.

In [ ]:
!pip install transformers torch pandas scikit-learn

In [ ]:
from google.colab import files
uploaded = files.upload()

import pandas as pd

# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/FAIRFRAME/only biasscore.csv')  # Replace with your actual file name
texts = data['text'].tolist()  # Adjust the column name as necessary
bias_scores = data['bias_score'].tolist()  # Adjust the column name as necessary

print(data.head())  # Display the first few rows of the dataset

                                                text  bias_score
0  NYPD Commissioner Dermot Shea on Monday expres...         0.0
1  School systems across the country are adopting...         9.5
2  And then along came President Barry Obama, who...         9.5
3  The curfews, which have never before occurred ...         6.0
4  Rather than help be a part of the solution, Tr...         8.5


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data
X_train, X_test, y_train, y_test = train_test_split(texts, bias_scores, test_size=0.2, random_state=42)

print(f'Training samples: {len(X_train)}, Testing samples: {len(X_test)}')

Training samples: 2496, Testing samples: 625


In [ ]:
from transformers import BertTokenizer
from torch.utils.data import Dataset, DataLoader

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class BiasDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = tokenizer(self.texts[idx], padding='max_length', truncation=True, return_tensors='pt', max_length=128)
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.float).unsqueeze(0)  # For regression
        }

# Create DataLoaders
train_dataset = BiasDataset(X_train, y_train)
test_dataset = BiasDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
import torch
from transformers import BertForSequenceClassification, AdamW

# Initialize the model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)  # 1 for regression
optimizer = AdamW(model.parameters(), lr=3e-5)

# Check if a GPU is available and move the model to the GPU if possible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
model.train()
for epoch in range(14):  # You can increase the number of epochs
    for batch in train_loader:
        optimizer.zero_grad()

        # Move batch to device
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

Epoch 1, Loss: 12.177310943603516
Epoch 2, Loss: 3.8858642578125
Epoch 3, Loss: 3.282750129699707
Epoch 4, Loss: 1.1737449169158936
Epoch 5, Loss: 0.8236576914787292
Epoch 6, Loss: 0.21999718248844147
Epoch 7, Loss: 0.8130311965942383
Epoch 8, Loss: 0.6198604106903076
Epoch 9, Loss: 0.24258381128311157
Epoch 10, Loss: 0.39190423488616943
Epoch 11, Loss: 0.19534790515899658
Epoch 12, Loss: 0.20017889142036438
Epoch 13, Loss: 0.41188132762908936
Epoch 14, Loss: 0.11943364143371582


In [ ]:
from sklearn.metrics import mean_squared_error

model.eval()
predictions = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = outputs.logits.squeeze().tolist()  # Get predictions

        if not isinstance(preds, list):
            preds = [preds]
        predictions.extend(preds)

# Calculate the Mean Squared Error
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 7.184906294263626


In [ ]:
path="/content/drive/MyDrive/FAIRFRAME/bias_score_model"
model.save_pretrained(path)
tokenizer.save_pretrained(path)

('/content/drive/MyDrive/FAIRFRAME/bias_score_model/tokenizer_config.json',
 '/content/drive/MyDrive/FAIRFRAME/bias_score_model/special_tokens_map.json',
 '/content/drive/MyDrive/FAIRFRAME/bias_score_model/vocab.txt',
 '/content/drive/MyDrive/FAIRFRAME/bias_score_model/added_tokens.json')

In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())  # Check if GPU is available


ValueError: module functions cannot set METH_CLASS or METH_STATIC

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, T5Tokenizer, T5ForConditionalGeneration


# Load the Bias Detection Model
bias_tokenizer = BertTokenizer.from_pretrained("/content/drive/MyDrive/FAIRFRAME/bias_score_model")
bias_model = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/FAIRFRAME/bias_score_model", num_labels=1)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bias_model.to(device)

# Load the Mitigation Model (T5)
mitigation_tokenizer = T5Tokenizer.from_pretrained("/content/drive/MyDrive/FAIRFRAME/mitigation_model")
mitigation_model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/FAIRFRAME/mitigation_model")
mitigation_model.to(device)

# Constants
max_input_length = 128
max_target_length = 128

def detect_bias(text):
    """Detects bias in the input text using the BERT model."""
    encoding = bias_tokenizer(text, padding='max_length', truncation=True, return_tensors='pt', max_length=128)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = bias_model(input_ids=input_ids, attention_mask=attention_mask)
        bias_score = outputs.logits.squeeze().item()

    return bias_score

def generate_unbiased_text(text):
    """Generates unbiased text using the T5 model."""
    input_ids = mitigation_tokenizer("unbias: " + text, return_tensors="pt", padding='max_length', max_length=max_input_length, truncation=True).input_ids
    input_ids = input_ids.to(device)

    with torch.no_grad():
        output = mitigation_model.generate(input_ids, max_length=max_target_length, num_beams=4, early_stopping=True)

    return mitigation_tokenizer.decode(output[0], skip_special_tokens=True)

# Get user input
user_input = input("Enter text: ")

# Detect bias
bias_score = detect_bias(user_input)
print("\nOriginal:", user_input)
#print("Bias Score:", bias_score)
# Check bias score and process accordingly
if bias_score > 3:
    intertext=generate_unbiased_text(user_input)
    new_score=detect_bias(intertext)
    i=0
    while(new_score>3 and i<=3):
        i+=1
        intertext=generate_unbiased_text(intertext)
        new_score=detect_bias(intertext)
        print("iteration:",i,"text:",intertext,"score:",new_score)
    print("unbiased text:",intertext)#,"\n new bias score",new_score''')
else:
    print("Low bias, no need to unbias.")

Enter text: # Simple larger text input in Colab text = input("Enter text:

Original: # Simple larger text input in Colab text = input("Enter text:
Low bias, no need to unbias.


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, T5Tokenizer, T5ForConditionalGeneration


# Load the Bias Detection Model
bias_tokenizer = BertTokenizer.from_pretrained("/content/drive/MyDrive/FAIRFRAME/bias_score_model")
bias_model = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/FAIRFRAME/bias_score_model", num_labels=1)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bias_model.to(device)

# Load the Mitigation Model (T5)
mitigation_tokenizer = T5Tokenizer.from_pretrained("/content/drive/MyDrive/FAIRFRAME/mitigation_model")
mitigation_model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/FAIRFRAME/mitigation_model")
mitigation_model.to(device)

# Constants
max_input_length = 128
max_target_length = 128

def detect_bias(text):
    """Detects bias in the input text using the BERT model."""
    encoding = bias_tokenizer(text, padding='max_length', truncation=True, return_tensors='pt', max_length=128)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = bias_model(input_ids=input_ids, attention_mask=attention_mask)
        bias_score = outputs.logits.squeeze().item()

    return bias_score

def generate_unbiased_text(text):
    """Generates unbiased text using the T5 model."""
    input_ids = mitigation_tokenizer("unbias: " + text, return_tensors="pt", padding='max_length', max_length=max_input_length, truncation=True).input_ids
    input_ids = input_ids.to(device)

    with torch.no_grad():
        output = mitigation_model.generate(input_ids, max_length=max_target_length, num_beams=4, early_stopping=True)

    return mitigation_tokenizer.decode(output[0], skip_special_tokens=True)

# Get user input
user_input = input("Enter text: ")

# Detect bias
bias_score = detect_bias(user_input)
print("\nOriginal:", user_input)
#print("Bias Score:", bias_score)
# Check bias score and process accordingly
if bias_score > 3:
    intertext=generate_unbiased_text(user_input)
    new_score=detect_bias(intertext)
    i=0
    while(new_score>3):
        i+=1
        intertext=generate_unbiased_text(intertext)
        new_score=detect_bias(intertext)
        print("iteration:",i,"text:",intertext,"score:",new_score)
    print("unbiased text:",intertext)#,"\n new bias score",new_score''')
else:
    print("Low bias, no need to unbias.")

Enter text: The corrupt politicians only care about lining their own pockets while ignoring the suffering of ordinary people.

Original: The corrupt politicians only care about lining their own pockets while ignoring the suffering of ordinary people.
iteration: 1 text: Politicians who hold power often place their financial interests above the well-being of their constituents. score: 4.705848693847656
iteration: 2 text: Politicians who hold power often prioritize their financial interests over the well-being of their constituents. score: 4.3238067626953125
iteration: 3 text: Politicians who hold power often place financial interests above the well-being of their constituents. score: 4.872226238250732
iteration: 4 text: Politicians who hold power often prioritize financial interests over the well-being of their constituents. score: 4.456850528717041
iteration: 5 text: Politicians who hold power often place financial interests above the well-being of their constituents. score: 4.872226238

KeyboardInterrupt: 

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, T5Tokenizer, T5ForConditionalGeneration


# Load the Bias Detection Model
bias_tokenizer = BertTokenizer.from_pretrained("/content/drive/MyDrive/FAIRFRAME/bias_score_model")
bias_model = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/FAIRFRAME/bias_score_model", num_labels=1)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bias_model.to(device)

# Load the Mitigation Model (T5)
mitigation_tokenizer = T5Tokenizer.from_pretrained("/content/drive/MyDrive/FAIRFRAME/mitigation_model")
mitigation_model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/FAIRFRAME/mitigation_model")
mitigation_model.to(device)

# Constants
max_input_length = 128
max_target_length = 128

def detect_bias(text):
    """Detects bias in the input text using the BERT model."""
    encoding = bias_tokenizer(text, padding='max_length', truncation=True, return_tensors='pt', max_length=128)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = bias_model(input_ids=input_ids, attention_mask=attention_mask)
        bias_score = outputs.logits.squeeze().item()

    return bias_score

def generate_unbiased_text(text):
    """Generates unbiased text using the T5 model."""
    input_ids = mitigation_tokenizer("unbias: " + text, return_tensors="pt", padding='max_length', max_length=max_input_length, truncation=True).input_ids
    input_ids = input_ids.to(device)

    with torch.no_grad():
        output = mitigation_model.generate(input_ids, max_length=max_target_length, num_beams=4, early_stopping=True)

    return mitigation_tokenizer.decode(output[0], skip_special_tokens=True)

import ipywidgets as widgets
from IPython.display import display

# Create a large text area
text_area = widgets.Textarea(
    placeholder="Enter your text here...",
    layout=widgets.Layout(width="100%", height="200px")  # Adjust size
)

display(text_area)

# Button to get input
button = widgets.Button(description="Submit")

output = widgets.Output()

def on_button_click(b):
    with output:
        output.clear_output()
        user_input = text_area.value
        #print("\nUser Input:", user_input)

        # Detect bias
        bias_score = detect_bias(user_input)
        print("\nOriginal:", user_input)
        print("\nBias Score:", bias_score)
        # Check bias score and process accordingly
        if bias_score > 3:
            intertext=generate_unbiased_text(user_input)
            new_score=detect_bias(intertext)
            i=0
            while(new_score>3 and i<=3):
                i+=1
                intertext=generate_unbiased_text(intertext)
                new_score=detect_bias(intertext)
                #print("iteration:",i,"text:",intertext,"score:",new_score)
            print("\nunbiased text:",intertext)#,"\n new bias score",new_score''')
        else:
            print("Low bias, no need to unbias.")

button.on_click(on_button_click)

display(button, output)


Textarea(value='', layout=Layout(height='200px', width='100%'), placeholder='Enter your text here...')

Button(description='Submit', style=ButtonStyle())

Output()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def compute_similarity(text1, text2):
    # Create a TF-IDF vectorizer
    vectorizer = TfidfVectorizer()

    # Fit and transform the input texts
    tfidf_matrix = vectorizer.fit_transform([text1, text2])

    # Compute cosine similarity
    similarity_score = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])

    return similarity_score[0][0]

# Example input
if bias_score>3:
  text1 = user_input
  text2 = intertext
  similarity = compute_similarity(text1, text2)
  print(f"Cosine Similarity Score: {similarity:.4f}")
else:
  print("no need to compare")

# Compute similarity


Cosine Similarity Score: 0.2435


In [ ]:
pip install transformers torch rouge-score nltk textstat


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 60.0 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24986 sha256=ed885edaa4c0c819491e30062cd20a8b390005bc60e3ee6bf633a5a911586912
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.6.4
    Uninstalling importlib-metadata-4.6.4:
      Successfully uninstalled importlib-metadata-4.6.4


In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import textstat
import numpy as np

# Load fine-tuned T5 model
model_name = "/content/drive/MyDrive/FAIRFRAME/mitigation_model"  # Update path if needed
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Sample test dataset (replace with actual dataset)
test_set = val_texts_only  # Ensure `val_texts_only` is defined

# Function to generate mitigated text using T5
def generate_mitigated_text(text):
    if not text.strip():
        return ""  # Handle empty inputs
    input_ids = tokenizer(text, return_tensors="pt", padding=True, truncation=True).input_ids.to(device)
    output_ids = model.generate(input_ids, max_new_tokens=100)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Function to calculate ROUGE scores
def calculate_rouge(reference, generated):
    scorer = rouge_scorer.RougeScorer(['rouge-1', 'rouge-2', 'rouge-l'], use_stemmer=True)
    return scorer.score(reference, generated)

# Function to calculate BLEU score with smoothing
def calculate_bleu(reference, generated):
    reference_tokens = reference.split()
    generated_tokens = generated.split()
    if not reference_tokens or not generated_tokens:
        return 0  # Avoid issues with empty text
    return sentence_bleu([reference_tokens], generated_tokens, smoothing_function=SmoothingFunction().method1)

# Function to calculate Flesch Reading Ease Score
def calculate_flesch_score(text):
    return textstat.flesch_reading_ease(text) if text.strip() else 0

# Lists to store scores
rouge_1_scores, rouge_2_scores, rouge_l_scores = [], [], []
bleu_scores = []
flesch_original_scores, flesch_mitigated_scores = [], []

# Process test set
for original_text in test_set:
    mitigated_text = generate_mitigated_text(original_text)

    rouge_scores = calculate_rouge(original_text, mitigated_text)
    rouge_1_scores.append(rouge_scores['rouge-1'].fmeasure)
    rouge_2_scores.append(rouge_scores['rouge-2'].fmeasure)
    rouge_l_scores.append(rouge_scores['rouge-l'].fmeasure)

    bleu_scores.append(calculate_bleu(original_text, mitigated_text))

    flesch_original_scores.append(calculate_flesch_score(original_text))
    flesch_mitigated_scores.append(calculate_flesch_score(mitigated_text))

# Compute average scores
average_rouge_1 = np.mean(rouge_1_scores)
average_rouge_2 = np.mean(rouge_2_scores)
average_rouge_l = np.mean(rouge_l_scores)
average_bleu = np.mean(bleu_scores)
average_flesch_original = np.mean(flesch_original_scores)
average_flesch_mitigated = np.mean(flesch_mitigated_scores)

# Print results
print(f"Average ROUGE-1 Score: {average_rouge_1:.4f}")
print(f"Average ROUGE-2 Score: {average_rouge_2:.4f}")
print(f"Average ROUGE-L Score: {average_rouge_l:.4f}")
print(f"Average BLEU Score: {average_bleu:.4f}")
print(f"Average Flesch Reading Ease Score (Original): {average_flesch_original:.2f}")
print(f"Average Flesch Reading Ease Score (Mitigated): {average_flesch_mitigated:.2f}")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


ValueError: Invalid rouge type: rouge-1

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import textstat
import numpy as np

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import textstat
import numpy as np

def print_debug_info(test_set):
    print("Debug Information:")
    print(f"Number of texts in test set: {len(test_set)}")
    if test_set:
        print("First few texts:")
        for i, text in enumerate(test_set[:3], 1):
            print(f"{i}. Text length: {len(text)} characters")
            print(f"   Preview: {text[:100]}...")

def evaluate_text_mitigation(model, tokenizer, test_set, device):
    # Print debug info first
    print_debug_info(test_set)

    # Validation checks
    if not test_set:
        print("Error: Test set is empty!")
        return None

    # Prepare lists to store results
    rouge_1_scores, rouge_2_scores, rouge_l_scores = [], [], []
    bleu_scores = []
    flesch_original_scores, flesch_mitigated_scores = [], []

    # Scorer for ROUGE
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    # Process each text
    for idx, original_text in enumerate(test_set, 1):
        try:
            # Ensure text is not empty
            if not original_text or not original_text.strip():
                print(f"Skipping empty text at index {idx}")
                continue

            # Tokenize and generate
            inputs = tokenizer(
                original_text,
                max_length=512,
                return_tensors="pt",
                truncation=True,
                padding=True
            ).to(device)

            # Generate mitigated text
            outputs = model.generate(
                inputs.input_ids,
                max_length=100,
                num_return_sequences=1
            )
            mitigated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Calculate ROUGE scores
            rouge_scores = scorer.score(original_text, mitigated_text)
            rouge_1_scores.append(rouge_scores['rouge1'].fmeasure)
            rouge_2_scores.append(rouge_scores['rouge2'].fmeasure)
            rouge_l_scores.append(rouge_scores['rougeL'].fmeasure)

            # Calculate BLEU score
            reference_tokens = original_text.split()
            generated_tokens = mitigated_text.split()
            bleu_score = sentence_bleu(
                [reference_tokens],
                generated_tokens,
                smoothing_function=SmoothingFunction().method1
            )
            bleu_scores.append(bleu_score)

            # Calculate Flesch scores
            flesch_original_scores.append(textstat.flesch_reading_ease(original_text) or 0)
            flesch_mitigated_scores.append(textstat.flesch_reading_ease(mitigated_text) or 0)

            # Print progress
            if idx % 10 == 0:
                print(f"Processed {idx} texts...")

        except Exception as e:
            print(f"Error processing text at index {idx}: {e}")

    # Compute and print results
    print("\nEvaluation Results:")
    print(f"ROUGE-1: {np.mean(rouge_1_scores):.4f}")
    print(f"ROUGE-2: {np.mean(rouge_2_scores):.4f}")
    print(f"ROUGE-L: {np.mean(rouge_l_scores):.4f}")
    print(f"BLEU Score: {np.mean(bleu_scores):.4f}")
    print(f"Flesch (Original): {np.mean(flesch_original_scores):.2f}")
    print(f"Flesch (Mitigated): {np.mean(flesch_mitigated_scores):.2f}")

    return {
        'rouge1': np.mean(rouge_1_scores),
        'rouge2': np.mean(rouge_2_scores),
        'rougeL': np.mean(rouge_l_scores),
        'bleu': np.mean(bleu_scores),
        'flesch_original': np.mean(flesch_original_scores),
        'flesch_mitigated': np.mean(flesch_mitigated_scores)
    }

# Actual usage
try:
    # Ensure these variables are defined before running
    model_name = "/content/drive/MyDrive/FAIRFRAME/mitigation_model"

    # Load model and tokenizer
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)

    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Ensure test_set is defined
    if 'val_texts_only' not in locals():
        print("Error: val_texts_only is not defined. Please define your test set.")
    else:
        # Run evaluation
        results = evaluate_text_mitigation(model, tokenizer, val_texts_only, device)
except Exception as e:
    print(f"Unexpected error: {e}")

Debug Information:
Number of texts in test set: 1893
First few texts:
1. Text length: 105 characters
   Preview: The blue whale Balaenoptera musculus is a marine mammal belonging to the baleen whale parvorder Myst...
2. Text length: 85 characters
   Preview: There are no historical references to an Aurelius Ursicinus in Britain in this period...
3. Text length: 212 characters
   Preview: His publishing house the Ten Bamboo Studio produced reference works on calligraphy poetry and art me...
Processed 10 texts...
Processed 20 texts...
Processed 30 texts...
Processed 40 texts...
Processed 50 texts...
Processed 60 texts...
Processed 70 texts...
Processed 80 texts...
Processed 90 texts...
Processed 100 texts...
Processed 110 texts...
Processed 120 texts...
Processed 130 texts...
Processed 140 texts...
Processed 150 texts...
Processed 160 texts...
Processed 170 texts...
Processed 180 texts...
Processed 190 texts...
Processed 200 texts...
Processed 210 texts...
Processed 220 texts...
Processe

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import numpy as np

# Batch size for processing
BATCH_SIZE = 16

def print_debug_info(test_set):
    print("Debug Information:")
    print(f"Total texts in test set: {len(test_set)}")
    if test_set:
        print("First few texts:")
        for i, text in enumerate(test_set[:3], 1):
            print(f"{i}. Text length: {len(text)} characters")
            print(f"   Preview: {text[:100]}...")

def batch_evaluate(model, tokenizer, test_set, device, batch_size=BATCH_SIZE):
    print_debug_info(test_set)

    if not test_set:
        print("Error: Test set is empty!")
        return None

    rouge_1_scores, rouge_2_scores, rouge_l_scores = [], [], []
    bleu_scores = []
    flesch_original_scores, flesch_mitigated_scores = [], []

    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    # Process in batches
    for i in range(0, len(test_set), batch_size):
        batch_texts = test_set[i:i + batch_size]

        try:
            # Ensure batch is not empty
            batch_texts = [text.strip() for text in batch_texts if text.strip()]
            if not batch_texts:
                continue

            # Tokenize and generate mitigated texts as batch
            inputs = tokenizer(batch_texts, max_length=512, return_tensors="pt", truncation=True, padding=True).to(device)
            outputs = model.generate(inputs.input_ids, max_length=100, num_return_sequences=1)
            mitigated_texts = [tokenizer.decode(out, skip_special_tokens=True) for out in outputs]

            # Calculate Scores for each text in the batch
            for original_text, mitigated_text in zip(batch_texts, mitigated_texts):
                rouge_scores = scorer.score(original_text, mitigated_text)
                rouge_1_scores.append(rouge_scores['rouge1'].fmeasure)
                rouge_2_scores.append(rouge_scores['rouge2'].fmeasure)
                rouge_l_scores.append(rouge_scores['rougeL'].fmeasure)

                reference_tokens = original_text.split()
                generated_tokens = mitigated_text.split()
                bleu_scores.append(sentence_bleu([reference_tokens], generated_tokens, smoothing_function=SmoothingFunction().method1))

                flesch_original_scores.append(textstat.flesch_reading_ease(original_text) or 0)
                flesch_mitigated_scores.append(textstat.flesch_reading_ease(mitigated_text) or 0)

            # Display batch-wise results
            print(f"\nProcessed Batch {i//batch_size + 1}:")
            print(f"  Average ROUGE-1: {np.mean(rouge_1_scores):.4f}")
            print(f"  Average ROUGE-2: {np.mean(rouge_2_scores):.4f}")
            print(f"  Average ROUGE-L: {np.mean(rouge_l_scores):.4f}")
            print(f"  Average BLEU Score: {np.mean(bleu_scores):.4f}")
            print(f"  Flesch (Original): {np.mean(flesch_original_scores):.2f}")
            print(f"  Flesch (Mitigated): {np.mean(flesch_mitigated_scores):.2f}")

        except Exception as e:
            print(f"Error processing batch {i//batch_size + 1}: {e}")

    # Compute final results
    print("\nFinal Evaluation Results:")
    print(f"Overall ROUGE-1: {np.mean(rouge_1_scores):.4f}")
    print(f"Overall ROUGE-2: {np.mean(rouge_2_scores):.4f}")
    print(f"Overall ROUGE-L: {np.mean(rouge_l_scores):.4f}")
    print(f"Overall BLEU Score: {np.mean(bleu_scores):.4f}")
    print(f"Overall Flesch (Original): {np.mean(flesch_original_scores):.2f}")
    print(f"Overall Flesch (Mitigated): {np.mean(flesch_mitigated_scores):.2f}")

    return {
        'rouge1': np.mean(rouge_1_scores),
        'rouge2': np.mean(rouge_2_scores),
        'rougeL': np.mean(rouge_l_scores),
        'bleu': np.mean(bleu_scores),
        'flesch_original': np.mean(flesch_original_scores),
        'flesch_mitigated': np.mean(flesch_mitigated_scores)
    }

# Actual execution
try:
    model_name = "/content/drive/MyDrive/FAIRFRAME/mitigation_model"

    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    if 'val_texts_only' not in locals():
        print("Error: val_texts_only is not defined. Please define your test set.")
    else:
        results = batch_evaluate(model, tokenizer, val_texts_only, device)
except Exception as e:
    print(f"Unexpected error: {e}")



Debug Information:
Total texts in test set: 1893
First few texts:
1. Text length: 105 characters
   Preview: The blue whale Balaenoptera musculus is a marine mammal belonging to the baleen whale parvorder Myst...
2. Text length: 85 characters
   Preview: There are no historical references to an Aurelius Ursicinus in Britain in this period...
3. Text length: 212 characters
   Preview: His publishing house the Ten Bamboo Studio produced reference works on calligraphy poetry and art me...

Processed Batch 1:
  Average ROUGE-1: 0.6862
  Average ROUGE-2: 0.5289
  Average ROUGE-L: 0.6242
  Average BLEU Score: 0.2598
  Flesch (Original): 40.54
  Flesch (Mitigated): 45.32

Processed Batch 2:
  Average ROUGE-1: 0.6927
  Average ROUGE-2: 0.5204
  Average ROUGE-L: 0.6326
  Average BLEU Score: 0.2775
  Flesch (Original): 42.01
  Flesch (Mitigated): 46.58

Processed Batch 3:
  Average ROUGE-1: 0.7096
  Average ROUGE-2: 0.5352
  Average ROUGE-L: 0.6541
  Average BLEU Score: 0.2965
  Flesch (Original

In [ ]:

import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from sentence_transformers import SentenceTransformer, util
import numpy as np

# Load a sentence embedding model for cosine similarity
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Batch size for processing
BATCH_SIZE = 16

def batch_evaluate_cosine(model, tokenizer, test_set, device, batch_size=BATCH_SIZE):
    if not test_set:
        print("Error: Test set is empty!")
        return None

    cosine_similarities = []

    # Process in batches
    for i in range(0, len(test_set), batch_size):
        batch_texts = test_set[i:i + batch_size]

        try:
            batch_texts = [text.strip() for text in batch_texts if text.strip()]
            if not batch_texts:
                continue

            # Tokenize and generate mitigated texts as batch
            inputs = tokenizer(batch_texts, max_length=512, return_tensors="pt", truncation=True, padding=True).to(device)
            outputs = model.generate(inputs.input_ids, max_length=100, num_return_sequences=1)
            mitigated_texts = [tokenizer.decode(out, skip_special_tokens=True) for out in outputs]

            # Compute embeddings for cosine similarity
            original_embeddings = embedding_model.encode(batch_texts, convert_to_tensor=True)
            mitigated_embeddings = embedding_model.encode(mitigated_texts, convert_to_tensor=True)
            batch_cosine_similarities = util.paired_cosine_sim(original_embeddings, mitigated_embeddings).cpu().numpy()

            cosine_similarities.extend(batch_cosine_similarities)

            # Display batch-wise results
            print(f"\nBatch {i//batch_size + 1}:")
            print(f"  Average Cosine Similarity: {np.mean(batch_cosine_similarities):.4f}")

        except Exception as e:
            print(f"Error processing batch {i//batch_size + 1}: {e}")

    # Compute final cosine similarity result
    overall_cosine_similarity = np.mean(cosine_similarities)
    print(f"\nOverall Cosine Similarity: {overall_cosine_similarity:.4f}")

    return overall_cosine_similarity

# Actual execution
try:
    model_name = "/content/drive/MyDrive/FAIRFRAME/mitigation_model"

    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    if 'val_texts_only' not in locals():
        print("Error: val_texts_only is not defined. Please define your test set.")
    else:
        cosine_similarity_score = batch_evaluate_cosine(model, tokenizer, val_texts_only, device)
except Exception as e:
    print(f"Unexpected error: {e}")
